In [8]:
import os
from os import listdir
from os.path import isfile, join

# cwd = os.getcwd() + '/dataInFiles' 
cwd = os.getcwd() + '/dataTemp2' 
onlyfiles = [os.path.join(cwd, f) for f in os.listdir(cwd) if 
os.path.isfile(os.path.join(cwd, f))]
print(onlyfiles) 

['/home/haidar-linux-laptop/programmingLinux/dalas_weather/meteo_scrap/dataTemp2/H_984_previous-2020-2024.csv.gz', '/home/haidar-linux-laptop/programmingLinux/dalas_weather/meteo_scrap/dataTemp2/H_77_previous-2020-2024.csv.gz', '/home/haidar-linux-laptop/programmingLinux/dalas_weather/meteo_scrap/dataTemp2/H_84_previous-2020-2024.csv.gz']


In [9]:
import gzip
import csv
import psycopg2
from datetime import datetime, timezone
from zoneinfo import ZoneInfo

conn = psycopg2.connect(
    dbname="dalas",
    user="postgres",
    password="mysecretpassword",
    host="localhost",
)

cur = conn.cursor()

for file in onlyfiles:
    with gzip.open(file, 'rt') as f:
        print(f'opening file {file}')
        conn.commit()

        reader = csv.DictReader(f, delimiter=';')
        for i, row in enumerate(reader):
            # print(f'{row['NUM_POSTE']} {row['LAT']} {row['LON']} {row['AAAAMMJJHH']} {row['RR1']} {row['VV']} {row['SOL']} {row['SOLNG']}')
            lat = float(row['LAT'])
            long = float(row['LON'])
            dateRaw = row['AAAAMMJJHH']
            dt = datetime.strptime(dateRaw, "%Y%m%d%H").replace(tzinfo=timezone.utc)
            dt_fr = dt.astimezone(ZoneInfo("Europe/Paris"))
            start = datetime(2022, 1, 1, tzinfo=timezone.utc)
            end   = datetime(2024, 1, 1, tzinfo=timezone.utc) 

            if start <= dt_fr < end:
                percipitation = row['RR1']
                if not percipitation:
                    percipitation = -1
                else:
                    percipitation = float(percipitation)
                visibility = row['VV']
                if not visibility:
                    visibility = -1
                else:
                    visibility = float(visibility)
                sol = row['SOL']
                if not sol:
                    sol = 'inconnu'
                solng = row['SOLNG']
                if not solng:
                    solng = 'inconnu'
                numPoste = row['NUM_POSTE']
                

                cur.execute("""
                INSERT INTO scrap (timestamp, percipitation, visibility, sol, solng, num_poste, location)
                VALUES (%s, %s, %s, %s, %s, %s,  ST_SetSRID(ST_MakePoint(%s, %s), 4326))
                """,
                (dt_fr, percipitation, visibility, sol, solng, numPoste, long, lat))

            if i % 100000 == 0:
                conn.commit()
                print(f'currently on {file} in data {i}')

conn.commit()

print('Finished!')
cur.close()
conn.close()

opening file /home/haidar-linux-laptop/programmingLinux/dalas_weather/meteo_scrap/dataTemp2/H_984_previous-2020-2024.csv.gz
currently on /home/haidar-linux-laptop/programmingLinux/dalas_weather/meteo_scrap/dataTemp2/H_984_previous-2020-2024.csv.gz in data 0
currently on /home/haidar-linux-laptop/programmingLinux/dalas_weather/meteo_scrap/dataTemp2/H_984_previous-2020-2024.csv.gz in data 100000
currently on /home/haidar-linux-laptop/programmingLinux/dalas_weather/meteo_scrap/dataTemp2/H_984_previous-2020-2024.csv.gz in data 200000
currently on /home/haidar-linux-laptop/programmingLinux/dalas_weather/meteo_scrap/dataTemp2/H_984_previous-2020-2024.csv.gz in data 300000
opening file /home/haidar-linux-laptop/programmingLinux/dalas_weather/meteo_scrap/dataTemp2/H_77_previous-2020-2024.csv.gz
currently on /home/haidar-linux-laptop/programmingLinux/dalas_weather/meteo_scrap/dataTemp2/H_77_previous-2020-2024.csv.gz in data 0
currently on /home/haidar-linux-laptop/programmingLinux/dalas_weather

In [10]:
cwd = os.getcwd() + '/dataInFiles' 
onlyfiles = [os.path.join(cwd, f) for f in os.listdir(cwd) if 
os.path.isfile(os.path.join(cwd, f))]
print(len(onlyfiles)) 

cwd = os.getcwd() + '/dataTemp' 
onlyfiles = [os.path.join(cwd, f) for f in os.listdir(cwd) if 
os.path.isfile(os.path.join(cwd, f))]
print(len(onlyfiles)) 

cwd = os.getcwd() + '/dataTemp2' 
onlyfiles = [os.path.join(cwd, f) for f in os.listdir(cwd) if 
os.path.isfile(os.path.join(cwd, f))]
print(len(onlyfiles)) 

96
6
3


In [24]:
import sys
import csv
import threading
import os.path
from datetime import datetime
from zoneinfo import ZoneInfo
from queue import Queue
import psycopg2
from psycopg2.extras import RealDictCursor

result_queue = Queue(maxsize=1000)  # prevents memory explosion
# accidents_2022_2023_meteo.csv

def run(csvFileName: str):
    with open(csvFileName, 'r') as csvFile:
        reader = csv.DictReader(csvFile, delimiter=';')
        rows = list(reader)

        threads = []
        resultFileName = 'result.csv'
        writer_thread = threading.Thread(target=writeToCSV, daemon=True, args=(resultFileName, ))
        writer_thread.start()


        for row in rows:
            t = threading.Thread(target=worker,args=(row,))
            threads.append(t)
            t.start()

            while threading.active_count() > 10:
                pass

            
        for t in threads:
            t.join()

        result_queue.put(None)
        writer_thread.join()
        

def worker(row):
    conn = psycopg2.connect(
        dbname="dalas",
        user="postgres",
        password="mysecretpassword",
        host="localhost",
    )

    cur = conn.cursor(cursor_factory=RealDictCursor)
    lat = float(row["lat"].replace(",", "."))
    lon = float(row["long"].replace(",", "."))
    hour = row['hrmn']
    date = f"{row['an']}-{row['mois']}-{row['jour']}"
    fullDate = f"{date} {hour}"
    id = row['Num_Acc']
    numPoste = row['com']

    dt = datetime.strptime(
        fullDate, "%Y-%m-%d %H:%M"
    ).replace(tzinfo=ZoneInfo("Europe/Paris"))    

    query = """
    SELECT *
    FROM scrap
    WHERE timestamp = %s AND num_poste = %s
    LIMIT 1;
    """
    # query = """
    # SELECT *
    # FROM scrap
    # WHERE timestamp = date_trunc('hour', %s)
    # ORDER BY
    # ST_Distance(
    #     location::geography,
    #     ST_SetSRID(ST_MakePoint(%s, %s), 4326)::geography
    # )
    # LIMIT 1
    # """

    cur.execute(
        query,
        (
            dt,        # datetime (tz-aware)
            numPoste
        )
    )
    # cur.execute(
    #     query,
    #     (
    #         dt,        # datetime (tz-aware)
    #         lon,       # longitude
    #         lat        # latitude
    #     )
    # )

    row = cur.fetchone()
    print(row)
    if(row):
        result_queue.put({'result': row, 'date': dt, "id": id})

def writeToCSV(fileName):
    written = 0
    has_header = os.path.isfile(f'{fileName}')
    with open (f'{fileName}.csv', 'a+', newline='') as csvfile:
        fieldNames = ['Num_Acc','date','percipitation', 'visibility', 'sol', 'solng']
        writer = csv.DictWriter(csvfile, fieldnames=fieldNames)
        if(not has_header):
            writer.writeheader()
        while True:
            item = result_queue.get()
            if item is None:
                break
            writer.writerow({'Num_Acc': item['id'],'date': item['date'], 'percipitation': item['result']['percipitation'], 'visibility': item['result']['visibility'],'sol': item['result']['sol'], 'solng': item['result']['solng']})
            result_queue.task_done()
            written +=1
            if(written % 100 == 0):
                print(f'Queue number {written}')
        return None

if __name__ == '__main__':
    fileName = '2022_23_pour_meteo.csv'
    run(fileName)
    # csvFile = openCSVFile(fileName)


None


KeyboardInterrupt: 

In [ ]:
import sys
import csv
import threading
import os.path
from datetime import datetime, timezone
from zoneinfo import ZoneInfo
from queue import Queue
import psycopg2
from psycopg2.extras import RealDictCursor

result_queue = Queue(maxsize=1000)  # prevents memory explosion
# accidents_2022_2023_meteo.csv

def run(csvFileName: str):
    with open(csvFileName, 'r') as csvFile:
        reader = csv.DictReader(csvFile, delimiter=';')
        rows = list(reader)

        threads = []
        resultFileName = 'result.csv'
        writer_thread = threading.Thread(target=writeToCSV, daemon=True, args=(resultFileName, ))
        writer_thread.start()


        for row in rows:
            worker(row)
        

def worker(row):
    conn = psycopg2.connect(
        dbname="dalas",
        user="postgres",
        password="mysecretpassword",
        host="localhost",
    )

    cur = conn.cursor(cursor_factory=RealDictCursor)
    lat = float(row["lat"].replace(",", "."))
    lon = float(row["long"].replace(",", "."))
    hour = row['hrmn']
    date = f"{row['an']}-{row['mois']}-{row['jour']}"
    fullDate = f"{date} {hour}"
    id = row['Num_Acc']
    numPoste = row['com']

    dt = datetime.strptime(
        fullDate, "%Y-%m-%d %H:%M"
    ).replace(tzinfo=timezone.utc, minute=0, second=0, microsecond=0)    

    query = """
    SELECT *
        FROM scrap
        WHERE timestamp = %s
        ORDER BY
        location <-> ST_SetSRID(ST_MakePoint(%s, %s), 4326)
        LIMIT 1;
    """

    cur.execute(
        query,
        (
            dt,        # datetime (tz-aware)
            lon,
            lat
        )
    )
    # cur.execute(
    #     query,
    #     (
    #         dt,        # datetime (tz-aware)
    #         lon,       # longitude
    #         lat        # latitude
    #     )
    # )

    print(dt)
    print(numPoste)
    row = cur.fetchone()
    print(row)
    if(row):
        result_queue.put({'result': row, 'date': dt, "id": id})

def writeToCSV(fileName):
    written = 0
    has_header = os.path.isfile(f'{fileName}')
    with open (f'{fileName}.csv', 'a+', newline='') as csvfile:
        fieldNames = ['Num_Acc','date','percipitation', 'visibility', 'sol', 'solng']
        writer = csv.DictWriter(csvfile, fieldnames=fieldNames)
        if(not has_header):
            writer.writeheader()
        while True:
            item = result_queue.get()
            if item is None:
                break
            writer.writerow({'Num_Acc': item['id'],'date': item['date'], 'percipitation': item['result']['percipitation'], 'visibility': item['result']['visibility'],'sol': item['result']['sol'], 'solng': item['result']['solng']})
            result_queue.task_done()
            written +=1
            if(written % 1000 == 0):
                print(f'Queue number {written}')
        return None

if __name__ == '__main__':
    fileName = '2022_23_pour_meteo.csv'
    run(fileName)
    # csvFile = openCSVFile(fileName)


2022-10-19 16:00:00+00:00
26198
RealDictRow({'id': 'aa5a5c77-cac7-491f-925b-b06da6247692', 'timestamp': datetime.datetime(2022, 10, 19, 16, 0, tzinfo=datetime.timezone.utc), 'percipitation': 0.0, 'visibility': 20000.0, 'sol': '0', 'solng': 'inconnu', 'location': '0101000020E61000003BDF4F8D97EE1240D94125AE634A4640', 'num_poste': '26198001'})
2022-10-20 08:00:00+00:00
25204
RealDictRow({'id': 'ba73a12f-2833-4e9f-b4ad-598f5169d402', 'timestamp': datetime.datetime(2022, 10, 20, 8, 0, tzinfo=datetime.timezone.utc), 'percipitation': 0.0, 'visibility': -1.0, 'sol': 'inconnu', 'solng': 'inconnu', 'location': '0101000020E610000048E17A14AE4719402DB29DEFA7764740', 'num_poste': '25462001'})
2022-10-20 17:00:00+00:00
22360
RealDictRow({'id': 'eea5baf8-dd0e-4e2b-8f0f-c20ed1623519', 'timestamp': datetime.datetime(2022, 10, 20, 17, 0, tzinfo=datetime.timezone.utc), 'percipitation': 0.0, 'visibility': 20000.0, 'sol': '0', 'solng': 'inconnu', 'location': '0101000020E61000002236583849D306C0E9263108AC4448

KeyboardInterrupt: 

In [7]:
import csv
import os
import threading
from datetime import datetime, timezone
from queue import Queue
from concurrent.futures import ThreadPoolExecutor

import psycopg2
from psycopg2.extras import RealDictCursor


# =========================
# CONFIGURATION
# =========================

DB_CONFIG = {
    "dbname": "dalas",
    "user": "postgres",
    "password": "mysecretpassword",
    "host": "localhost",
}

INPUT_CSV = "2022_23_pour_meteo.csv"
OUTPUT_CSV = "result.csv"

MAX_WORKERS = 10          # good for 6c / 12t CPU
QUEUE_MAXSIZE = 1000


# =========================
# SHARED QUEUE
# =========================

result_queue = Queue(maxsize=QUEUE_MAXSIZE)


# =========================
# WORKER FUNCTION
# =========================

def worker(row: dict):
    """
    One task = one DB query.
    Each worker gets its own DB connection.
    """

    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor(cursor_factory=RealDictCursor)

    try:
        # --- Parse coordinates (French CSV format)
        lat = float(row["lat"].replace(",", "."))
        lon = float(row["long"].replace(",", "."))

        # --- Build datetime (rounded to hour, UTC)
        date = f"{row['an']}-{row['mois']}-{row['jour']}"
        full_date = f"{date} {row['hrmn']}"

        dt = datetime.strptime(
            full_date, "%Y-%m-%d %H:%M"
        ).replace(
            minute=0,
            second=0,
            microsecond=0,
            tzinfo=timezone.utc
        )

        accident_id = row["Num_Acc"]

        # --- Fast spatial nearest-neighbor query
        query = """
        SELECT *
        FROM scrap
        WHERE timestamp = %s
        ORDER BY
            location <-> ST_SetSRID(ST_MakePoint(%s, %s), 4326)
        LIMIT 1;
        """

        cur.execute(query, (dt, lon, lat))
        result = cur.fetchone()

        if result:
            result_queue.put({
                "Num_Acc": accident_id,
                "date": dt,
                "percipitation": result["percipitation"],
                "visibility": result["visibility"],
                "sol": result["sol"],
                "solng": result["solng"],
            })

    except Exception as e:
        print("Worker error:", e)

    finally:
        cur.close()
        conn.close()


# =========================
# CSV WRITER THREAD
# =========================

def write_to_csv():
    written = 0
    file_exists = os.path.isfile(OUTPUT_CSV)

    with open(OUTPUT_CSV, "a", newline="") as csvfile:
        fieldnames = [
            "Num_Acc",
            "date",
            "percipitation",
            "visibility",
            "sol",
            "solng",
        ]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        if not file_exists:
            writer.writeheader()

        while True:
            item = result_queue.get()
            if item is None:
                break

            writer.writerow(item)
            written += 1

            if written % 1000 == 0:
                print(f"Wrote {written} rows")

            result_queue.task_done()


# =========================
# MAIN DRIVER
# =========================

def run():
    # --- Load CSV
    with open(INPUT_CSV, "r") as f:
        reader = csv.DictReader(f, delimiter=";")
        rows = list(reader)

    print(f"Loaded {len(rows)} rows")

    # --- Start writer thread
    writer_thread = threading.Thread(target=write_to_csv, daemon=True)
    writer_thread.start()

    # --- Thread pool for DB workers
    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        for row in rows:
            executor.submit(worker, row)

    # --- Stop writer
    result_queue.put(None)
    writer_thread.join()

    print("Done.")


# =========================
# ENTRY POINT
# =========================

if __name__ == "__main__":
    run()


Loaded 110124 rows
Wrote 1000 rows
Wrote 2000 rows
Wrote 3000 rows
Wrote 4000 rows
Wrote 5000 rows
Wrote 6000 rows
Wrote 7000 rows
Wrote 8000 rows
Wrote 9000 rows
Wrote 10000 rows
Wrote 11000 rows
Wrote 12000 rows
Wrote 13000 rows
Wrote 14000 rows
Wrote 15000 rows
Wrote 16000 rows
Wrote 17000 rows
Wrote 18000 rows
Wrote 19000 rows
Wrote 20000 rows
Wrote 21000 rows
Wrote 22000 rows
Wrote 23000 rows
Wrote 24000 rows
Wrote 25000 rows
Wrote 26000 rows
Wrote 27000 rows
Wrote 28000 rows
Wrote 29000 rows
Wrote 30000 rows
Wrote 31000 rows
Wrote 32000 rows
Wrote 33000 rows
Wrote 34000 rows
Wrote 35000 rows
Wrote 36000 rows
Wrote 37000 rows
Wrote 38000 rows
Wrote 39000 rows
Wrote 40000 rows
Wrote 41000 rows
Wrote 42000 rows
Wrote 43000 rows
Wrote 44000 rows
Wrote 45000 rows
Wrote 46000 rows
Wrote 47000 rows
Wrote 48000 rows
Wrote 49000 rows
Wrote 50000 rows
Wrote 51000 rows
Wrote 52000 rows
Wrote 53000 rows
Wrote 54000 rows
Wrote 55000 rows
Wrote 56000 rows
Wrote 57000 rows
Wrote 58000 rows
Wrot